# Storing skymaps

## Read raw skymap files

In [2]:
import pickle
from pathlib import Path
import math
import numpy as np
from lsst.sphgeom import Box, ConvexPolygon, LonLat, UnitVector3d
import yaml
from skymap_to_poly_coords import load_pickle_skymap


In [3]:
package_root = Path.home() / "skymap-to-poly-coords"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

skymap_out_dir = package_root / "skymaps_out"
inner_poly_path = skymap_out_dir / "inner_polygons.yaml"
outer_poly_path = skymap_out_dir / "outer_polygons.yaml"

In [4]:
lsst_skymap = load_pickle_skymap(skymap_path)
lsst_skymap

## A note on rings sky map pixelization 
*(in html comment in this cell)*
<!---## Rings sky map pixelization
*From the RingsSkyMap docstring in lsst.skymap:*

We divide the sphere into N rings of Declination, plus the two polar
caps, which sets the size of the individual tracts.  The rings are
divided in RA into an integral number of tracts of this size; this
division is made at the Declination closest to zero so as to ensure
full overlap.

Rings are numbered in the rings from south to north. The south pole cap is
``tract=0``, then the tract at ``raStart`` in the southernmost ring is
``tract=1``. Numbering continues (in the positive RA direction) around that
ring and then continues in the same fashion with the next ring north, and
so on until all reaching the north pole cap, which is
``tract=len(skymap) - 1``.

However, ``version=0`` had a bug in the numbering of the tracts: the first
and last tracts in the first (southernmost) ring were identical, and the
first tract in the last (northernmost) ring was missing. When using
``version=0``, these tracts remain missing in order to preserve the
numbering scheme.--->

## Storage options


We would like to support both inner and outer polygons for the tracts.  

Tracts are arranged in "rings", which span horizontal regions of the sky.

→ Inner polys:
- represent the exact boundaries of a given tract
- do not overlap

→ Outer polys:
- may (will?) overlap with adjacent tracts' outer polys
- are akin to margins in HATS catalogs

## Option 1: Two YAML files (inner polys and outer polys) that store each tracts' corners

#### Inner polys/outer polys:
- Store per-tract: `ra_min`, `ra_max`, `dec_min`, `dec_max`
- Comes directly from `getRaDecRange(tract_index)`
- Requires 4 floats per tract, so for ~19k tracts:
  4 × 8 B × 19,000 ≈ ~600 KB (before YAML overhead; ~1–1.5 MB total)
- However, could compress to a `.npz` of around ~0.5–0.8 MB

#### Pros
- Easy to understand and verify
- Doesn’t require any LSST WCS machinery
- Reading is fast + LSST-free

#### Cons
- Disk usage
- Less precise because of floating points?
- Storing patches is gonna go crazy on this. Better to iterate to a more performant version for that.

### Write inner_poly and outer_poly files

In [4]:
from skymap_to_poly_coords import write_polygons_ra_dec


write_polygons_ra_dec(lsst_skymap, inner_poly_path, inner=True)
write_polygons_ra_dec(lsst_skymap, outer_poly_path, inner=False)

## Option 2: Reconstruct everything using projection + geometry

### Things to store:
#### Global:
  - `projection: "TAN"`
  - `pixel_scale_deg`: (or arcsec/pixel)
  - `tract_width`, `tract_height`: in pixels
  - `overlap_deg`: or overlap in pixels (need this for inner bounds)

#### Per-tract:
  - `tract_id`
  - `center_ra`, `center_dec`
  - `rotation` (optional, usually 0)
  - `flipX` (optional, for WCS handedness)

### Then, you reconstruct inner and outer polys by:
- Creating a fake `SkyWcs` from these params
- Converting pixel corners of `[0,0]` → `[width,height]` to sky coordinates
- Optionally applying overlap trimming in pixel space

### Pros
- Super compact: ~2 floats + a few constants per tract
- Elegant reuse of LSST projection machinery
- One unified format

### Cons
- Requires LSST-style WCS math to reconstruct accurately
- Makes assumptions about projection type and tract uniformity
- Harder to read/debug

## Option 3: hybrid; store per-ring Dec data, and per-tract RA data
### Similar principle with patches--each row of patches will act as its own "patch ring" with shared Dec

Can we make this assumption, or does it break down with projection distortions near the poles, rounding errors, or other confounding factors?

### Let's confirm two things:
1. Are the dec_min and dec_max values exactly the same for all tracts in a given ring?
2. Can each tract polygon be reconstructed from just the ring’s declination bounds and the tract’s RA vertex list?

### Assumption 1: dec_min and dec_max are exactly the same for all tracts in a ring (✓)
(within floating point imprecision)

In [10]:
import yaml
from pathlib import Path

yaml_path = inner_poly_path  # todo check outers also

# Load YAML
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

# Parse and sort tracts
tract_items = sorted(data["tracts"].items(), key=lambda x: int(x[0]))

rings = []
current_ring = []
last_dec_bounds = None

for tract_id_str, entry in tract_items:
    tract_id = int(tract_id_str)
    polygon = entry.get("polygon")

    if polygon is None:
        print(f"Skipping degenerate tract {tract_id} with no polygon data.")
        continue

    decs = [point[1] for point in polygon]

    # Collect rounded decs to eliminate fp imprecision
    unique_decs = {round(d, 12) for d in decs}
    if len(unique_decs) > 2:
        raise ValueError(f"⚠️ Tract {tract_id} has {len(unique_decs)} unique Decs: {sorted(unique_decs)}")

    # Generate bounds for declination
    dec_min = round(min(decs), 12)
    dec_max = round(max(decs), 12)
    dec_bounds = (dec_min, dec_max)

    if last_dec_bounds is None:
        # First ring
        last_dec_bounds = dec_bounds

    if dec_bounds != last_dec_bounds:
        # New ring starts
        rings.append((last_dec_bounds, current_ring))
        current_ring = [tract_id]
        last_dec_bounds = dec_bounds
    else:
        current_ring.append(tract_id)

# Append the last ring
if current_ring:
    rings.append((last_dec_bounds, current_ring))

# Report
verbose = False
if verbose:
    for i, (bounds, tracts) in enumerate(rings):
        print(f"→ Ring {i}: dec_min/max = {bounds}, tracts = {tracts[0]} to {tracts[-1]}, total = {len(tracts)}")
else:
    print(f"→ Found {len(rings)} rings with {sum(len(r[1]) for r in rings)} tracts.")

→ Found 122 rings with 18938 tracts.


### Assumption 2: Each tract polygon can be reconstructed from just the ring’s declination bounds and the tract’s RA vertex list